# Feed-forward neural net. 

This notebook is meant to be run **online**, in google colab to take advantage of the free GPUs. If you run the data reading cell locally, it might yak stuff up in your local directory. 

Simplifications I'm making FOR NOW: 
- No punctuation
- No test data

In [ ]:
# TODO re-tokenize

In [138]:
import torch, spacy
import random
import numpy as np
from torch import nn
from tqdm.notebook import trange, tqdm
import os


from matplotlib import pyplot as plt
from torch import optim
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_ as clip_grad_norm
from torch.nn import functional as F

seed = 811

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False
print(torch.randn(5))

tensor([-1.1484,  0.0960, -2.0328, -0.0796, -0.7443])


In [139]:
#Read in data...

!rm -rf processed_data processed_data.zip
import urllib.request
url = "https://github.com/benartuso/bias-detection/blob/main/data/processed_data.zip?raw=true"
filename, headers = urllib.request.urlretrieve(url, filename="processed_data.zip")
!unzip processed_data.zip
print("Done")

Archive:  processed_data.zip
   creating: processed_data/
  inflating: processed_data/dev_lstm.csv  
  inflating: processed_data/biased.word.dev  
  inflating: processed_data/full_dev.csv  
  inflating: processed_data/dev_ann.csv  
  inflating: processed_data/train_ann.csv  
  inflating: processed_data/biased.word.test  
  inflating: processed_data/test_ann.csv  
  inflating: processed_data/full_test.csv  
  inflating: processed_data/biased.word.train  
  inflating: processed_data/full_train.csv  
  inflating: processed_data/train_lstm.csv  
  inflating: processed_data/test_lstm.csv  
Done


In [5]:
# !pip install transformers --quiet
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


     |████████████████████████████████| 1.4MB 17.6MB/s 
     |████████████████████████████████| 2.9MB 53.8MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 


In [134]:
from torchtext.data import Field, ReversibleField, Dataset, TabularDataset, BucketIterator, Iterator
from spacy.tokenizer import Tokenizer

spacy_en = spacy.load('en')

# def tokenize_fn(text):
#   return text.strip().split()

def reader(train_name, dev_name, suffix = ".csv", rpath="processed_data", batch_size=8, min_freq=2):
  #Text field, sequence data
  TXT = Field(sequential=True, tokenize="spacy", init_token=None, eos_token=None, lower=True)
  #Binary labels
  LABEL = Field(sequential=False, unk_token=None, dtype=torch.long, use_vocab=False)

  #Create Dataset
  fields = [("text", TXT), ("label", LABEL)]
  trn_data = TabularDataset(os.path.join(rpath, train_name+suffix), format="CSV", fields=fields, skip_header=False)
  val_data = TabularDataset(os.path.join(rpath, dev_name+suffix), format="CSV", fields=fields, skip_header=False)


  #Split
  #Build vocab on train set  
  TXT.build_vocab(trn_data, min_freq=min_freq)

  train_iter, val_iter = BucketIterator.splits((trn_data, val_data), 
                                               batch_size=batch_size, 
                                               sort_key= lambda x: len(x.text),
                                               shuffle=False,
                                               repeat=False)
  
  return train_iter, val_iter, TXT

In [135]:
train_iter, val_iter, txtfield = reader("full_train", "full_dev", suffix=".csv", min_freq=5, batch_size=16)
vocab_size = len(txtfield.vocab)

print("Vocab size = {}".format(vocab_size))
pad = txtfield.vocab.stoi[txtfield.pad_token]

print("[TRAIN]:%d (dataset:%d)\t[VAL]:%d (dataset:%d)\t"
    % (len(train_iter), len(train_iter.dataset),
    len(val_iter), len(val_iter.dataset)))
print("[vocab]:%d" % (vocab_size))

Vocab size = 52691
[TRAIN]:19282 (dataset:308504)	[VAL]:1135 (dataset:18147)	
[vocab]:52691


In [136]:
txtfield.vocab.load_vectors('glove.6B.50d')

In [141]:
txtfield.vocab.vectors.shape

torch.Size([52691, 50])

In [33]:
# # https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

# import gensim.downloader
# print(*gensim.downloader.info()['models'].keys(), sep="\n")

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [34]:
# glove_vectors = gensim.downloader.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [41]:
# glove_vocab = glove_vectors.vocab.keys()

In [47]:
# glove_vectors.word_vec(".")

array([ 1.5164e-01,  3.0177e-01, -1.6763e-01,  1.7684e-01,  3.1719e-01,
        3.3973e-01, -4.3478e-01, -3.1086e-01, -4.4999e-01, -2.9486e-01,
        1.6608e-01,  1.1963e-01, -4.1328e-01, -4.2353e-01,  5.9868e-01,
        2.8825e-01, -1.1547e-01, -4.1848e-02, -6.7989e-01, -2.5063e-01,
        1.8472e-01,  8.6876e-02,  4.6582e-01,  1.5035e-02,  4.3474e-02,
       -1.4671e+00, -3.0384e-01, -2.3441e-02,  3.0589e-01, -2.1785e-01,
        3.7460e+00,  4.2284e-03, -1.8436e-01, -4.6209e-01,  9.8329e-02,
       -1.1907e-01,  2.3919e-01,  1.1610e-01,  4.1705e-01,  5.6763e-02,
       -6.3681e-05,  6.8987e-02,  8.7939e-02, -1.0285e-01, -1.3931e-01,
        2.2314e-01, -8.0803e-02, -3.5652e-01,  1.6413e-02,  1.0216e-01],
      dtype=float32)

## Basic FFNN + CNN

In [142]:
class NeuralClassifier(nn.Module):
    def __init__(self, vocab, vocab_size, embed_size, drop_rate=0.0, class_size=2, pad=1):
        super(NeuralClassifier, self).__init__()
        """ Initialization
        - vocab_size
        - embed_size: word embedding size
        - drop_rate: dropout rate
        - class_size: number of classes. For binary classification, class_size = 2
        """
        # ---------------------------------
        # Configuration
        self.vocab_size = vocab_size # size of the vocab
        self.class_size = class_size # number of classes
        self.using_cuda = True
        self.dropout = nn.Dropout(drop_rate)
        self.pad = pad
        # ---------------------------------
        # Network parameters
        # vocab.vectors = vocab.vectors.to("cuda:0")
        self.embed = nn.Embedding.from_pretrained(vocab.vectors, padding_idx=pad)
        self.fc = nn.Linear(embed_size, class_size, bias=True)


    def forward(self, batch):
        """ Forward function
        """
        input, label = batch.text.to("cuda:0"), batch.label.to("cuda:0")
        
        # ---------------------------------
        # === Hidden layer ===
        # Sum over all the embeddings for each input text
        #   then, pass through the nonlinear Sigmoid function
        x = self.embed(input) # Dim: L x B x E
        x = self.dropout(x)
        hidden = torch.sigmoid(x.sum(axis=0)) # Dim: B x E
        
        # ---------------------------------
        # === Classification layer ===
        logit = self.fc(hidden) # Dim: Batch_size x Class_size
        # Normalization
        logprob = F.log_softmax(logit, dim=1)
        
        # ---------------------------------
        # === Loss function ===
        # Compute negative log-likelihood loss
        loss = F.cross_entropy(logprob, label)
        return loss, logprob


In [147]:
class SimpleCNN(nn.Module):
    def __init__(self, vocab, vocab_size, embed_size, drop_rate=0.0,
                 class_size=2, kernel_sizes=[2,3,4],
                 dropout=0.0, using_cuda=False, pad=None):
        super(SimpleCNN, self).__init__()
        # ---------------------------------
        # Configuration
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.class_size = class_size # number of classes
        self.kernel_sizes = kernel_sizes # a list of kernel sizes
        self.using_cuda = using_cuda
        self.pad = pad # index of <pad> in vocab

        # not sure if should include
        self.dropout = nn.Dropout(drop_rate)
        # ---------------------------------
        # Model Arch
        self.embed = nn.Embedding.from_pretrained(vocab.vectors, padding_idx=pad)
        self.conv2 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[0])
        self.conv3 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[1])
        self.conv4 = torch.nn.Conv1d(in_channels=embed_size, out_channels=embed_size, kernel_size=self.kernel_sizes[2])
        self.tanh = torch.tanh
        self.maxpool = torch.max
        self.fc = nn.Linear(embed_size*3, class_size, bias=True)


    def forward(self, batch):
        # ===============================
        # Implement the CNN specified in the homework
        input, label = batch.text.to("cuda:0"), batch.label.to("cuda:0")

        # initial input
        x0 = self.embed(input)
        x0 = self.dropout(x0) # L X B X E
        x0 = torch.transpose(x0, 0, 2) # -> E X B X L
        x0 = torch.transpose(x0, 0, 1) # -> B X E X L

        # through convolutional layer
        x1_0 = self.conv2(x0)
        x1_1 = self.conv3(x0)
        x1_2 = self.conv3(x0)

        # tanh activation function
        x2_0 = self.tanh(x1_0)
        x2_1 = self.tanh(x1_1)
        x2_2 = self.tanh(x1_2)

        # max pooling
        x3_0 = self.maxpool(x2_0, dim=2).values # dim=2 is L dimension, -> B X E
        x3_1 = self.maxpool(x2_1, dim=2).values
        x3_2 = self.maxpool(x2_2, dim=2).values

        # concatenating/preparing for softmax
        x4 = torch.cat([x3_0, x3_1, x3_2], dim=1) # -> B X 3E
        logit = self.fc(x4) # B X class_size

        # ===============================
        
        logprob = F.log_softmax(logit, dim=1)
        loss = F.cross_entropy(logprob, label)
        return loss, logprob

In [65]:

def batch_train(batch, model, optimizer):
    """ Training with one batch
    - batch: a min-batch of the data
    - model: the defined neural network
    - optimizer: optimization method used to update the parameters
    """
    # set in training mode
    model.train()
    # initialize optimizer
    optimizer.zero_grad()
    # forward: prediction
    loss, _ = model(batch)
    # backward: gradient computation
    loss.backward()
    # norm clipping, in case the gradient norm is too large
    clip_grad_norm(model.parameters(), grad_clip)
    # gradient-based update parameter
    optimizer.step()
    return model, loss.item()

In [76]:
def eval(data_iter, model):
    """ Evaluate the model with the data
    data_iter: the data iterator 
    model: the defined model
    """
    # set in the eval model, which will trun off the features only used for training, such as droput
    model.eval()
    # records
    val_loss, val_batch = 0, 0
    total_example, correct_pred = 0, 0
    # iterate all the mini batches for evaluation
    for b, batch in enumerate(data_iter):
        # Forward: prediction
        loss, logprob = model(batch)
        # 
        val_batch += 1
        val_loss += loss
        # Argmax
        max_logprob, pred_label = torch.max(logprob, -1)
        correct_pred += (pred_label==batch.label.to("cuda:0")).sum()
        total_example += batch.label.size()[0]
    acc = (1.0*correct_pred)/total_example
    # print("val_batch = {}".format(val_batch))
    return (val_loss/val_batch), acc

In [ ]:
torch.cuda.set_device("cuda:0")


In [82]:
txtfield.vocab.stoi["<pad>"]

1

In [177]:
model = NeuralClassifier(txtfield.vocab, vocab_size, embed_size=50, drop_rate=0.5, class_size=2, pad=1)
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0, momentum=0.5)
# the norm of grad clipping
grad_clip = 1.0

# ------------------------------------
# 3. Define the numbers of training epochs and validation steps
epoch, val_step = 10, 5000

# ------------------------------------
# 4. Training iterations
TrnLoss, ValLoss, ValAcc = [], [], []
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        model, loss = batch_train(batch, model, optimizer)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc = eval(val_iter, model)
            ValLoss.append(val_loss)
            ValAcc.append(val_acc)
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc)))

0.5279660820960999 0.6919684410095215 0.6524286270141602
0.5349093675613403 0.690112829208374 0.7134592533111572
0.5352399945259094 0.6895871758460999 0.7512287497520447



0.5361768007278442 0.6892748475074768 0.6865637898445129
0.5368380546569824 0.6890871524810791 0.7103714346885681
0.5384361147880554 0.689092218875885 0.7038947343826294
0.5386013984680176 0.6889625191688538 0.6772952079772949



0.5395933389663696 0.6888644695281982 0.7134497761726379
0.5354052782058716 0.6889658570289612 0.6792764663696289
0.5407505631446838 0.6887090802192688 0.6674817800521851
0.5381605625152588 0.6889238953590393 0.6699913144111633



0.5383809804916382 0.6889857053756714 0.6564081311225891
0.5424588322639465 0.6887034773826599 0.7084526419639587
0.5407505631446838 0.6887551546096802 0.7028985023498535
0.5406403541564941 0.688880980014801 0.6692895889282227



0.5416322350502014 0.6887151002883911 0.6982148885726929
0.5427343249320984 0.6886809468269348 0.7206549048423767
0.5373340249061584 0.6892672777175903 0.6804261207580566
0.5350746512413025 0.6897716522216797 0.6929646730422974



0.5411362648010254 0.6886913180351257 0.7130241990089417
0.541411817073822 0.6888580918312073 0.7008991837501526
0.5397586226463318 0.6890396475791931 0.6952739953994751
0.5400892496109009 0.6887865662574768 0.7059463262557983



0.5379401445388794 0.6891434192657471 0.6797068119049072
0.5368380546569824 0.6893929839134216 0.6370939016342163
0.5400892496109009 0.6887905597686768 0.6433818936347961



0.5387667417526245 0.6890049576759338 0.707618772983551
0.5399239659309387 0.688683271408081 0.7092275023460388
0.5411913990974426 0.6886208653450012 0.6741013526916504
0.5402545928955078 0.6886962652206421 0.6998451948165894



0.5413566827774048 0.6886524558067322 0.6971451044082642
0.5374442338943481 0.6886049509048462 0.6790250539779663
0.539317786693573 0.6886656880378723 0.6764525175094604
0.5401443839073181 0.68879234790802 0.7131758332252502



0.5395933389663696 0.6888877153396606 0.7062641978263855
0.5373340249061584 0.6888614296913147 0.7146735787391663
0.5388769507408142 0.6887679696083069 0.7055890560150146
0.5363972187042236 0.689271092414856 0.639716386795044


The best validation accuracy = 0.5427


In [174]:
cnn = SimpleCNN(txtfield.vocab, vocab_size, embed_size=50, drop_rate=0.5, class_size=2, pad=1)
cnn.cuda()
optimizer_cnn = optim.SGD(cnn.parameters(), lr=0.001, weight_decay=0, momentum=0.9)
# the norm of grad clipping
grad_clip = 1.0

# ------------------------------------
# 3. Define the numbers of training epochs and validation steps
epoch, val_step = 10, 5000

# ------------------------------------
# 4. Training iterations
TrnLoss_cnn, ValLoss_cnn, ValAcc_cnn = [], [], []
total_batch = 0
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)
print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn))) # oops, messed up here...

0.5112690925598145 0.7023765444755554 0.6637200713157654
0.5513858795166016 0.687161386013031 0.635652482509613
0.5774508118629456 0.6753517389297485 0.7934437990188599



0.5626274347305298 0.6836695075035095 0.6322294473648071
0.5779467821121216 0.6760335564613342 0.7329779863357544
0.5746404528617859 0.6778451800346375 0.6919088363647461
0.5794897079467773 0.6751983165740967 0.6396294236183167



0.5821347832679749 0.6734665632247925 0.7444080710411072
0.5814183950424194 0.6734427809715271 0.6947196125984192
0.5880861878395081 0.6685365438461304 0.6911892890930176
0.5912271738052368 0.6669042706489563 0.5980028510093689



0.5896291136741638 0.6667019128799438 0.646432638168335
0.5895189046859741 0.6684778928756714 0.6725380420684814
0.590951681137085 0.6671911478042603 0.6552273035049438
0.5940926671028137 0.6653721928596497 0.6899318099021912



0.5840634703636169 0.671261727809906 0.6942598819732666
0.5922741889953613 0.6661866307258606 0.730685293674469
0.5928252339363098 0.6664373874664307 0.6583591103553772
0.5869840979576111 0.6682982444763184 0.7273115515708923



0.591282308101654 0.6652241945266724 0.6473812460899353
0.5951396822929382 0.662929117679596 0.6796454191207886
0.587755560874939 0.6670612692832947 0.7036731839179993
0.5943130850791931 0.6628369688987732 0.6905168294906616



0.5928252339363098 0.6640338897705078 0.5621476173400879
0.5900148749351501 0.6688314080238342 0.5669508576393127
0.59624183177948 0.662354052066803 0.5789520740509033



0.5951396822929382 0.6621959209442139 0.6422635316848755
0.5850002765655518 0.6679279208183289 0.6707759499549866
0.5940926671028137 0.6624585390090942 0.6670147180557251
0.5943130850791931 0.6623688340187073 0.7390958070755005



0.5846696496009827 0.6713070869445801 0.7099123597145081
0.5969581604003906 0.6627253293991089 0.6323556900024414
0.5948641896247864 0.6632276177406311 0.6659340262413025
0.592880368232727 0.664652943611145 0.6513873934745789



0.599437952041626 0.6613162755966187 0.6427916884422302
0.5964071154594421 0.6619261503219604 0.6615750193595886
0.5954703092575073 0.6631531119346619 0.6525360345840454
0.5926048159599304 0.6625378131866455 0.6222408413887024


The best validation accuracy = 0.5994


In [175]:
# what happens if we keep training? 
epoch = 10
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)

print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn))) # (printing the real max validation accuracy here)

0.5966826677322388 0.6623576283454895 0.7383233308792114
0.593982458114624 0.6642469763755798 0.6824289560317993
0.5956356525421143 0.6616234183311462 0.7072445750236511
0.598776638507843 0.6608774065971375 0.5891517996788025



0.596076488494873 0.6629798412322998 0.7627059817314148
0.5969030857086182 0.6604475378990173 0.7248205542564392
0.5954152345657349 0.6598755121231079 0.5802905559539795
0.5951396822929382 0.661114513874054 0.6558722853660583



0.592384397983551 0.6648821830749512 0.6925346851348877
0.594643771648407 0.6609976887702942 0.610667884349823
0.595580518245697 0.6609537601470947 0.6622121930122375
0.5953601002693176 0.6615242958068848 0.7193859219551086



0.5886372327804565 0.666978657245636 0.7032560706138611
0.5953601002693176 0.6619400382041931 0.6499238610267639
0.5992174744606018 0.6586481332778931 0.6647881269454956



0.5993277430534363 0.6586434245109558 0.60232013463974
0.5964071154594421 0.6601148247718811 0.6292406320571899
0.5929355025291443 0.6599704027175903 0.7103679180145264
0.5979501008987427 0.6587586402893066 0.5775989294052124



0.5997134447097778 0.6604812741279602 0.5992621779441833
0.5937620401382446 0.6634986400604248 0.6435559988021851
0.5966275334358215 0.6602556705474854 0.7254883646965027
0.5944232940673828 0.6631035208702087 0.6352949142456055



0.5910067558288574 0.6658790707588196 0.7021148204803467
0.5976194143295288 0.660334050655365 0.6211651563644409
0.5964071154594421 0.6620985865592957 0.6445168852806091
0.5974541306495667 0.6585530638694763 0.6163419485092163



0.5983358025550842 0.6591363549232483 0.7013702988624573
0.5988868474960327 0.6591762900352478 0.769411027431488
0.5998787879943848 0.6579678654670715 0.6357558965682983
0.5987215638160706 0.6586716771125793 0.5765960216522217



0.5976194143295288 0.659835934638977 0.6132401823997498
0.5942580103874207 0.6616050601005554 0.6537600755691528
0.6000992059707642 0.6576661467552185 0.8066065311431885
0.5944232940673828 0.6586773991584778 0.7196034789085388



0.5943682193756104 0.6588760614395142 0.7177297472953796
0.594147801399231 0.6614817976951599 0.5963104963302612
0.5981705188751221 0.659460186958313 0.7240805625915527
0.5917782783508301 0.6636928915977478 0.7164403796195984


The best validation accuracy = 0.6001


In [163]:
# what happens if we keep training? 
epoch = 10
for e in trange(epoch):
    # print(e)
    for b, batch in tqdm(enumerate(train_iter)):
        total_batch += 1
        # Update parameters with one batch
        cnn, loss = batch_train(batch, cnn, optimizer_cnn)
        # Compute validation loss after each val_step
        if total_batch % val_step == 0:
            val_loss, val_acc = eval(val_iter, cnn)
            ValLoss_cnn.append(val_loss.item())
            ValAcc_cnn.append(val_acc.item())
            print(val_acc.item(), val_loss.item(), loss)
            TrnLoss_cnn.append(loss)

print("The best validation accuracy = {:.4}".format(max(ValAcc_cnn))) # (printing the real max validation accuracy here)

0.6032401919364929 0.6528351902961731 0.702263355255127
0.6061608195304871 0.6546379327774048 0.6892585754394531
0.5991072654724121 0.6557237505912781 0.6628591418266296
0.6036810278892517 0.6534117460250854 0.6784582138061523



0.6051138043403625 0.6534462571144104 0.6065849661827087
0.6050035953521729 0.6517773866653442 0.5631462931632996
0.6028544902801514 0.6553735136985779 0.6195168495178223
0.6041219234466553 0.6531988978385925 0.8033552765846252



0.600374698638916 0.6543312668800354 0.6839261651039124
0.5955254435539246 0.6613097190856934 0.5645306706428528
0.6040116548538208 0.6528324484825134 0.6930787563323975
0.5922741889953613 0.6594828963279724 0.6658787131309509



0.5933763384819031 0.6672849655151367 0.6449357867240906
0.5981153845787048 0.6554303765296936 0.5885457992553711
0.6041219234466553 0.6533237099647522 0.6609498858451843
0.6027442812919617 0.6534786820411682 0.6544384956359863



0.5943682193756104 0.6579352021217346 0.8633385300636292
0.6022483110427856 0.654146134853363 0.7386988401412964
0.5991072654724121 0.6548939943313599 0.7000861167907715



0.6023033857345581 0.6552510261535645 0.6417559385299683
0.6041769981384277 0.6543204188346863 0.6307691931724548
0.59624183177948 0.658093273639679 0.6761047840118408
0.6033504009246826 0.6541323661804199 0.6356745362281799



0.6007053256034851 0.6584629416465759 0.5640724301338196
0.6021931767463684 0.6535690426826477 0.6133657097816467
0.6050035953521729 0.6530734300613403 0.6715427041053772
0.5993828177452087 0.6557624340057373 0.66584312915802



0.6029095649719238 0.6563785672187805 0.680713415145874
0.6032953262329102 0.6548580527305603 0.6800282597541809
0.5948641896247864 0.6592713594436646 0.6220301389694214
0.6053342223167419 0.6522029042243958 0.7154382467269897



0.605168879032135 0.653475284576416 0.668168306350708
0.5984460115432739 0.6596938967704773 0.6080683469772339
0.6023033857345581 0.6541016697883606 0.5925831198692322
0.601972758769989 0.6533316969871521 0.6338523030281067



0.6040116548538208 0.6567651629447937 0.6543886065483093
0.6052240133285522 0.6541005373001099 0.7623462677001953
0.5975092053413391 0.6564701199531555 0.5896580219268799
0.5891332030296326 0.6667397022247314 0.6162274479866028


The best validation accuracy = 0.6066
